In [2]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import LongType, TimestampType, StringType

StatementMeta(, c320daed-a08f-4725-b8ca-9cd467215d67, 4, Finished, Available, Finished)

In [3]:
df_activos = spark.sql("SELECT * FROM lakehouseSilver.dfactivobronze")

StatementMeta(, c320daed-a08f-4725-b8ca-9cd467215d67, 5, Finished, Available, Finished)

In [4]:
dim_categoria = spark.sql("""
    SELECT skCategoriaActivo, CAST(TRIM(codigocategoria) AS STRING) AS codigocategoria
    FROM lakehouseSilver.dimensionCategoriaSilver
""")

StatementMeta(, c320daed-a08f-4725-b8ca-9cd467215d67, 6, Finished, Available, Finished)

In [5]:
df_activos = df_activos.withColumn("codigocategoria", F.trim(F.col("codigocategoria").cast("string")))

StatementMeta(, c320daed-a08f-4725-b8ca-9cd467215d67, 7, Finished, Available, Finished)

In [6]:
df_final = df_activos.join(dim_categoria, on="codigocategoria", how="left")

StatementMeta(, c320daed-a08f-4725-b8ca-9cd467215d67, 8, Finished, Available, Finished)

In [7]:
try:
    df_sk_hist = spark.read.format("delta").table("lakehouseSilver.dimensionActivosSilver") \
        .select("idActivo", "skActivo")
    df_final = df_final.join(df_sk_hist, on="idActivo", how="left")
except:
    df_final = df_final.withColumn("skActivo", F.lit(None).cast("int"))

StatementMeta(, c320daed-a08f-4725-b8ca-9cd467215d67, 9, Finished, Available, Finished)

In [8]:
window_sk = Window.orderBy("idActivo")

StatementMeta(, c320daed-a08f-4725-b8ca-9cd467215d67, 10, Finished, Available, Finished)

In [9]:
df_final = df_final.withColumn(
    "skActivo",
    F.when(F.col("skActivo").isNull(), F.row_number().over(window_sk) + 200000) # Offset para diferenciar
     .otherwise(F.col("skActivo"))
)

StatementMeta(, c320daed-a08f-4725-b8ca-9cd467215d67, 11, Finished, Available, Finished)

In [10]:
df_final = df_final.withColumn("FechaCreacion", F.current_timestamp()) \
                   .withColumn("EsActual", F.lit(True))

StatementMeta(, c320daed-a08f-4725-b8ca-9cd467215d67, 12, Finished, Available, Finished)

In [11]:
if df_final.groupBy("skActivo").count().filter("count > 1").count() > 0:
    raise Exception("❌ Error: claves sustitutas duplicadas en skActivo")

StatementMeta(, c320daed-a08f-4725-b8ca-9cd467215d67, 13, Finished, Available, Finished)

In [12]:
df_final = df_final.select(
    "skActivo", "idActivo", "numContrato", "skCategoriaActivo", "FechaCreacion", "EsActual"
)

StatementMeta(, c320daed-a08f-4725-b8ca-9cd467215d67, 14, Finished, Available, Finished)

In [13]:
display(df_final)

StatementMeta(, c320daed-a08f-4725-b8ca-9cd467215d67, 15, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, fd743d55-5e5d-4aaa-b58c-0cac3c3f5dd1)

In [14]:
df_final.write.mode("overwrite").format("delta").saveAsTable("lakehouseSilver.dimensionActivosSilver")

StatementMeta(, c320daed-a08f-4725-b8ca-9cd467215d67, 16, Finished, Available, Finished)